## Preparación de datos para sistema recomendador proxy

En este *notebook* prepararemos los datos necesarios para evaluar un sistema recomendador externo con la librería `Elliot`. Se usarán los datasets de libros, usuarios y ratings.

### Creación de datasets de entrenamiento, validación y test

In [1]:
import os
import pandas as pd

# Ruta de datasets procesados
ready_path = os.path.join(os.getcwd(), "../..", "datasets", "ready")

# Lectura del dataset de ratings procesado
col_types = {
    'user_id': 'int32',
    'book_id': 'int32',
    'rating': 'float32',
}
ratings_df = pd.read_csv(ready_path + "/ratings.csv", dtype=col_types)
print("# de ratings:", ratings_df.shape[0])

# de ratings: 5515602


In [2]:
# Ruta de datasets en bruto
raw_path = os.path.join(os.getcwd(), "../..", "datasets", "raw")

# Lectura del dataset de libros en bruto (coincidirá con el procesado después de limpiarlo)
books_df = pd.DataFrame(pd.read_pickle(raw_path + "/books_raw.pkl"))
print("# de libros:", books_df.shape[0])

# de libros: 9467


In [3]:
# IDs de usuarios
user_ids = ratings_df['user_id'].unique()
print("# de usuarios:", len(user_ids))

# de usuarios: 52371


Debido a la gran cantidad de usuarios, seleccionamos 10000 de ellos aleatoriamente para un entrenamiento más rápido.

In [4]:
# Selección de num_users usuarios aleatorios
import random

random.seed(42)
num_users = 10000
user_ids_reduced = random.sample(list(user_ids), num_users)
print("# de usuarios:", len(user_ids_reduced))

# de usuarios: 10000


Ahora, el número de transacciones será más reducido, si bien lo suficientemente representativo, para el cómputo de las métricas de evaluación del sistema recomendador.

In [5]:
ratings_reduced_df = ratings_df[ratings_df['user_id'].isin(user_ids_reduced)]
print("# de ratings:", ratings_reduced_df.shape[0])

# de ratings: 1053374


Separamos el dataset de ratings en parte de entrenamiento y parte de test, en una proporción del 80% y 20%, respectivamente.

In [6]:
# Creamos los datasets de train (80%) y test (20%)
import sklearn.model_selection as model_selection

train, test = model_selection.train_test_split(ratings_reduced_df, test_size=0.2, random_state=42)
print("# de ratings en train (train + validation):", train.shape[0])
print("# de ratings en test:", test.shape[0])

# de ratings en train (train + validation): 842699
# de ratings en test: 210675


Además, en vista de modelos futuros, el dataset de entrenamiento será parte entrenamiento (70% del global) y parte validación (10% del global). Es fácil comprobar que se debe hacer un split local del dataset de entrenamiento en un 87.5% para entrenamiento *per se* y un 12.5% para validación, consiguiendo los porcentajes globales deseados.

In [7]:
# Creamos los datasets de train (87.5% -> 70% del global) y validación (12.5% -> 10% del global)
train_train, train_valid = model_selection.train_test_split(train, test_size=0.125, random_state=42)
print("# de ratings en train:", train_train.shape[0])
print("# de ratings en validation:", train_valid.shape[0])

# de ratings en train: 737361
# de ratings en validation: 105338


Ahora, vamos a calcular la matriz de similitudes entre los libros del dataset. Para ello, necesitamos crear una función de similitud. Como tanto los libros como los perfiles de usuario cuentan con los mismos atributos (`semantic_sbert`, `semantic_use` y `sentiment`) la función de similitud podrá utilizarse para comparar libros con libros, usuarios con usuarios o libros con usuarios (y viceversa). En este caso, nos interesará para ver la similitud entre libros.

La fórmula propuesta es la siguiente: $$S(x, y) = w * S_C(x_{sem}, y_{sem}) + (1 - w) * S_C(x_{sent}, y_{sent}),$$ donde $w \in [0, 1]$ es un peso para la ponderación entre la parte semántica y la parte de sentiment analysis y $S_C$ es la similitud coseno entre los vectores.

In [8]:
import numpy as np

# Definimos la función de similitud entre dos objetos
def sem_sent_sim(item1, item2, sem_option='semantic_sbert', sem_w=0.8) -> float:
    """
    Calcula la similitud entre un objeto y otro.
    Los objetos deben tener un vector `'semantic'` y otro `'sentiment'`, 
    previamente normalizados.

    La similitud sigue la siguiente fórmula:

    sim(item1, item2) = sem_w * cos_sim(item1[sem_option], item2[sem_option]) + 
    (1 - sem_w) * cos_sim(item1['sentiment'], item2['sentiment'])

    ## Parámetros:
    - item1: Primer objeto.
    - item2: Segundo objeto.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.8.

    ## Retorna:
    La similitud entre ambos objetos.
    """
    # Obtenemos el campo semántico de cada objeto
    sem1 = item1[sem_option]
    sem2 = item2[sem_option]
    # Obtenemos el campo de sentimiento de cada objeto
    sent1 = item1['sentiment']
    sent2 = item2['sentiment']
    # Calculamos la similitud
    return sem_w * np.dot(sem1, sem2) + (1 - sem_w) * np.dot(sent1, sent2)

def sem_sent_dist(item1, item2, sem_option='semantic_sbert', sem_w=0.8) -> float:
    """
    Calcula la distancia entre un objeto y otro.

    Los objetos deben tener un vector 'semantic' y otro 'sentiment', 
    previamente normalizados.

    La similitud sigue la siguiente fórmula:

    dist(item1, item2) = 1 - sim(item1, item2)

    ## Parámetros:
    - item1: Primer objeto.
    - item2: Segundo objeto.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.8.

    ## Retorna:
    La distancia entre ambos objetos.
    """
    return 1 - sem_sent_sim(item1, item2, sem_option, sem_w)

In [9]:
def sim_matrix(items, sem_option='semantic_sbert', sem_w=0.8):
    """
    Calcula la matriz de similitud entre los objetos de un dataset.
    
    ## Parámetros:
    - items: Dataset de objetos.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.8.
    
    ## Retorna:
    La matriz de similitud entre los objetos del dataset.
    """
    num_items = items.shape[0]
    matrix = np.zeros((num_items, num_items))

    for i in range(num_items):
        for j in range(num_items):
            item1 = items.iloc[i]
            item2 = items.iloc[j]
            matrix[i, j] = sem_sent_sim(item1, item2, sem_option, sem_w)

    return matrix

def matrix_to_df(matrix, items, item_id='book_id'):
    """
    Convierte una matriz de similitud en un dataframe.
    
    ## Parámetros:
    - matrix: Matriz de similitud.
    - items: Dataset de objetos.
    - item_id: Nombre de la columna que contiene el ID de cada objeto.
    
    ## Retorna:
    Un dataframe con la matriz de similitud.
    """
    return pd.DataFrame(matrix, index=items[item_id], columns=items[item_id])

Un ejemplo para 4 libros.

In [10]:
book_ids = [1, 12, 17, 20]
books = books_df[books_df['book_id'].isin(book_ids)]
similarity_matrix = sim_matrix(books, sem_w=0.8)
print(similarity_matrix)

[[0.9999999  0.61621011 0.70077719 0.71809572]
 [0.61621011 1.0000001  0.52647183 0.46321196]
 [0.70077719 0.52647183 1.00000019 0.7246611 ]
 [0.71809572 0.46321196 0.7246611  1.0000001 ]]


In [11]:
similarity_df = matrix_to_df(similarity_matrix, books)
similarity_df.loc[[17, 20], [1, 12]]

book_id,1,12
book_id,,
17,0.700777,0.526472
20,0.718096,0.463212


In [12]:
# Matriz de similitud entre libros
similarity_matrix = sim_matrix(books_df, sem_w=0.8)

In [17]:
similarity_df = matrix_to_df(similarity_matrix, books_df)
similarity_df.loc[[17, 20], [1, 12]]

book_id,1,12
book_id,,
17,0.700777,0.526472
20,0.718096,0.463212


Guardamos la matriz de similitud para uso futuro.

In [19]:
# Guarda la matriz de similitud
similarity_df.to_csv(ready_path + "/book_similarity.csv", index=True)

Podemos cargar la matriz directamente desde la siguiente celda sin necesidad de calcularla de nuevo.

In [27]:
# Load similarity matrix
similarity_df = pd.read_csv(ready_path + "/book_similarity.csv", index_col=0, dtype='float32')
similarity_df.index = similarity_df.index.astype('int16')
similarity_df.loc[[17, 20]]

,1,2,3,4,6,7,8,9,11,12,...,9963,9965,9968,9969,9971,9973,9982,9985,9987,10000
book_id,,,,,,,,,,,,,,,,,,,,,
17,0.700777,0.318825,0.193065,0.160345,0.228328,0.237620,0.346737,0.288061,0.263016,0.526472,...,0.380829,0.225062,0.254349,0.184396,0.479671,0.479671,0.469189,0.191034,0.525387,0.247396
20,0.718096,0.285862,0.131382,0.216830,0.209667,0.235295,0.350952,0.246060,0.281529,0.463212,...,0.338160,0.273511,0.206503,0.176313,0.422291,0.422291,0.413554,0.245042,0.444242,0.252896


### Recomendaciones del modelo (vector de similitudes)

Para las recomendaciones, nos basaremos en el historial de valoraciones de cada usuario. Se seleccionará, por cada usuario, una lista de libros que haya valorado positivamente. Esto es, con un rating mayor o igual que 0.75. Luego, con la matriz de similitudes se obtendrá la submatriz correspondiente a dichos libros y, teniendo en cuenta el rating, se calculará una suma por cada uno de los libros del dataset de dichas similitudes. De esta forma se obtendrá un vector de similitudes suma, del cual se escogerán aquellos libros con los $k$ valores mayores.

Si representamos dicha submatriz como

$$\begin{bmatrix}
S_{j_1;1} & S_{j_1;2} & \ldots & S_{j_1;N}\\
S_{j_2;1} & S_{j_2;2} & \ldots & S_{j_2;N} \\
\vdots & \vdots & \ddots & \vdots \\
S_{j_l;1} & S_{j_l;2} & \ldots & S_{j_l;N}
\end{bmatrix},
$$

donde $j_1, \ldots, j_l$ son los índices de los libros que ha valorado positivamente el usuario, el vector resultante es

$$
\begin{bmatrix}
\sum\limits_{i=1}^{l}r_{j_i}S_{j_i;1}, & \sum\limits_{i=1}^{l}r_{j_i}S_{j_i;2}, & \ldots, & \sum\limits_{i=1}^{l}r_{j_i}S_{j_i;N}
\end{bmatrix}
$$

Finalmente, se toman los libros que se correspondan con las $k$ coordenadas mayores de este vector. Nos referimos con $r_{j_i}$ a la valoración (positiva) que el usuario ha dado al libro $j_i$ para $i \in \{1, \ldots, l\}$.

In [29]:
# Creamos el dataframe de entrenamiento para el modelo a partir del dataset de train
train_df = pd.DataFrame(train, columns=['user_id', 'book_id', 'rating'])
train_df.sort_values(by=['user_id', 'book_id'], inplace=True)
train_df.reset_index(drop=True, inplace=True)
train_df.head()

,user_id,book_id,rating
0,8,13,0.75
1,8,14,1.00
2,8,28,0.75
3,8,43,0.25
4,8,45,0.50


In [30]:
# Creamos un dataframe con los usuarios que aparecen en el dataset de train
users_df = pd.DataFrame(columns=['user_id'])
users_df['user_id'] = train_df['user_id'].unique()
users_df.sort_values(by=['user_id'], inplace=True)
users_df.reset_index(drop=True, inplace=True)
print("# de usuarios:", users_df.shape[0])

# de usuarios: 10000


In [31]:
# Obtenemos la información de los libros que han sido valorados por los usuarios del dataset de train
books_df = books_df[books_df['book_id'].isin(train_df['book_id'].unique())]
books_df.sort_values(by=['book_id'], inplace=True)
books_df.reset_index(drop=True, inplace=True)
print("# de libros:", books_df.shape[0])

# de libros: 9467


In [56]:
def k_nearest(user_id: int, sim_matrix: pd.DataFrame, train_df: pd.DataFrame, books_df: pd.DataFrame, k=10):
    """
    Calcula los k libros más cercanos a un usuario.

    ## Parámetros:
    - user_id: ID del usuario del que queremos obtener los k libros más cercanos.
    - sim_matrix: Matriz de similitud entre libros.
    - train_df: `DataFrame` del dataset de entrenamiento.
    - books_df: `DataFrame` de libros en el dataset de entrenamiento.
    - k: Número de vecinos más cercanos que queremos obtener. Por defecto
    su valor es 10.

    ## Retorna:
    k tuplas (book_id, similitud) con los k vecinos más cercanos al objeto.
    """
    def sim_vector(sim_matrix: pd.DataFrame, user_likes: pd.DataFrame):
        """
        Calcula el vector de similitudes para los libros respecto de un usuario.

        ## Parámetros:
        - sim_matrix: Matriz de similitud entre libros.
        - user_likes: `DataFrame` de los libros que le gustan al usuario
        en el dataset de entrenamiento.

        ## Retorna:
        Vector con las similitudes suma.
        """
        sub_matrix = sim_matrix.loc[user_likes['book_id']]
        pond_sub_matrix = sub_matrix * user_likes['rating'].values.reshape(-1, 1)
        return np.sum(pond_sub_matrix, axis=0)
    
    # Ratings positivas del usuario
    user_ratings = train_df[train_df['user_id'] == user_id]
    user_likes = user_ratings[user_ratings['rating'] >= 0.75]
    sim_vec = sim_vector(sim_matrix, user_likes)
    nearest = list(zip(sim_matrix.index, sim_vec))
    # Quitar de la lista los libros que ya le gustan al usuario
    nearest = [x for x in nearest if x[0] not in user_ratings['book_id'].values]
    # Obtenemos los k vecinos más cercanos
    nearest.sort(key=lambda x: x[1], reverse=True)
    return nearest[:k]

Ejemplo para el usuario con identificador 8.

In [57]:
k_nearest(8, similarity_df, train_df, books_df)

[(3134, 18.33085823059082),
 (4769, 17.717296600341797),
 (1123, 17.69397735595703),
 (2953, 17.66826057434082),
 (6769, 17.596405029296875),
 (7384, 17.589773178100586),
 (689, 17.49903678894043),
 (2679, 17.470413208007812),
 (5734, 17.467529296875),
 (9216, 17.45812225341797)]

In [59]:
# Calculamos los k libros más similares a cada usuario (sin contar los ya positivamente valorados)
k = 50
predictions_df = users_df.copy()
predictions_df['nearest'] = predictions_df['user_id'].apply(lambda x: k_nearest(x, similarity_df, train_df, books_df, k=k))

In [60]:
predictions_df.head()

,user_id,nearest
0,8,"[(3134, 18.33085823059082), (4769, 17.71729660..."
1,13,"[(3134, 28.66318702697754), (3962, 28.39384651..."
2,14,"[(9216, 17.394306182861328), (7731, 17.1793060..."
3,20,"[(692, 27.253429412841797), (8785, 27.03939437..."
4,28,"[(3134, 29.49867057800293), (2088, 28.55392265..."


In [61]:
# Transformamos la columna 'nearest' en dos columnas: 'book_id' y 'prediction'
predictions_df = predictions_df.explode('nearest')
predictions_df[['book_id', 'prediction']] = pd.DataFrame(predictions_df['nearest'].tolist(), index=predictions_df.index)
predictions_df = predictions_df.drop('nearest', axis=1)

In [62]:
predictions_df.head()

,user_id,book_id,prediction
0,8,3134,18.330858
0,8,4769,17.717297
0,8,1123,17.693977
0,8,2953,17.668261
0,8,6769,17.596405


### Creación de los ficheros para Elliot

Una vez calculadas todas las predicciones de los libros por usuario a partir del modelo, guardaremos las propias predicciones en formato .tsv, de tal forma que puedan ser utilizadas en `Elliot` como `ProxyRecommender`.

In [63]:
# Ruta de datasets de entrenamiento
training_path = os.path.join(os.getcwd(), "../..", "datasets", "training")
# Guardamos el dataset de predicciones así como los de entrenamiento, test y validación
predictions_df.to_csv(training_path + "/predictions/predictions_cb.tsv", sep='\t', header=None, index=False)